In [1]:
import numpy as np
import matplotlib.pyplot as plt

from functions.pressure_conversion import pressure_conversion
from Ausgleichsbecken.Ausgleichsbecken_class_file import Ausgleichsbecken_class
from Druckrohrleitung.Druckrohrleitung_class_file import Druckrohrleitung_class
from Turbinen.Turbinen_class_file import Francis_Turbine

In [2]:
#define constants

#Turbine
Q_nenn          = 0.85  # m³/s
p_nenn          = pressure_conversion(10.6,'bar','Pa')
closing_time    = 5    #s

# physics
g               = 9.81                          # gravitational acceleration [m/s²]
rho             = 1000.                         # density of water [kg/m³]

# pipeline
L               = 535.+478.                     # length of pipeline [m]
D               = 0.9                           # pipe diameter [m]
A_pipe          = D**2/4*np.pi                  # pipeline area
h_pipe          = 105                           # hydraulic head without reservoir [m] 
alpha           = np.arcsin(h_pipe/L)           # Höhenwinkel der Druckrohrleitung 
n               = 50                            # number of pipe segments in discretization                  # initial flow velocity [m/s]
f_D             = 0.014                           # Darcy friction factor
c               = 400.                          # propagation velocity of the pressure wave [m/s]
# consider prescribing a total simulation time and deducting the number of timesteps from that
nt              = 9000                          # number of time steps after initial conditions

# derivatives of the pipeline constants
dx              = L/n                           # length of each pipe segment
dt              = dx/c                          # timestep according to method of characterisitics
nn              = n+1                           # number of nodes
initial_level   = 8.                            # water level in upstream reservoir [m]
pl_vec          = np.arange(0,nn,1)*dx          # pl = pipe-length. position of the nodes on the pipeline
t_vec           = np.arange(0,nt+1)*dt          # time vector
h_vec           = np.arange(0,nn,1)*h_pipe/n    # hydraulic head of pipeline at each node 



# reservoir
# replace influx by vector
initial_flux                = Q_nenn/1.1        # initial influx  of volume to the reservoir [m³/s]
initial_pressure_unit       = 'Pa'      # DO NOT CHANGE! for pressure conversion in print statements and plot labels 
conversion_pressure_unit    = 'bar'     # for pressure conversion in print statements and plot labels
area_base                   = 74.       # total base are of the cuboid reservoir [m²]   
area_outflux                = A_pipe    # outlfux area of the reservoir, given by pipeline area [m²]
critical_level_low          = 0.        # for yet-to-be-implemented warnings[m]
critical_level_high         = np.inf    # for yet-to-be-implemented warnings[m]

# make sure e-RK4 method of reservoir has a small enough timestep to avoid runaway numerical error
nt_eRK4                     = 100      # number of simulation steps of reservoir in between timesteps of pipeline              
simulation_timestep         = dt/nt_eRK4




In [3]:
# create objects

V = Ausgleichsbecken_class(area_base,area_outflux,critical_level_low,critical_level_high,simulation_timestep)
V.set_steady_state(initial_flux,initial_level,conversion_pressure_unit)


pipe = Druckrohrleitung_class(L,D,n,alpha,f_D)
pipe.set_pressure_propagation_velocity(c)
pipe.set_number_of_timesteps(nt)
pipe.set_steady_state(initial_flux,initial_level,area_base,pl_vec,h_vec)

initial_pressure_turbine = pipe.get_current_pressure_distribution()[-1]

T1 = Francis_Turbine(Q_nenn,p_nenn,closing_time,timestep=dt)
T1.set_steady_state(initial_flux,initial_pressure_turbine)


In [4]:
# initialization for timeloop

# prepare the vectors in which the pressure and velocity distribution in the pipeline from the previous timestep are stored
v_old = pipe.get_current_velocity_distribution()
p_old = pipe.get_current_pressure_distribution()

# prepare the vectors in which the temporal evolution of the boundary conditions are stored
    # keep in mind, that the velocity at the turbine and the pressure at the reservoir follow from boundary conditions
        # reservoir level and flow through turbine
    # the pressure at the turbine and the velocity at the reservoir are calculated from the method of characteristics
v_boundary_res  = np.zeros_like(t_vec)
v_boundary_tur  = np.zeros_like(t_vec)
p_boundary_res  = np.zeros_like(t_vec)
p_boundary_tur  = np.zeros_like(t_vec)

# prepare the vectors that store the temporal evolution of the level in the reservoir
level_vec       = np.full(nt+1,initial_level)  # level at the end of each pipeline timestep

# set the boundary conditions for the first timestep
v_boundary_res[0]   = v_old[0]
v_boundary_tur[0]   = v_old[-1] 
p_boundary_res[0]   = p_old[0]
p_boundary_tur[0]   = p_old[-1]

LA_soll_vec = np.full_like(t_vec,T1.LA)
# LA_soll_vec[500:]= 0
LA_ist_vec  = np.full_like(t_vec,T1.LA)




In [5]:
%matplotlib qt5
# time loop

# create a figure and subplots to display the velocity and pressure distribution across the pipeline in each pipeline step
fig1,axs1 = plt.subplots(2,1)
fig1.suptitle(str(0) +' s / '+str(round(t_vec[-1],2)) + ' s' )
axs1[0].set_title('Pressure distribution in pipeline')
axs1[1].set_title('Velocity distribution in pipeline')
axs1[0].set_xlabel(r'$x$ [$\mathrm{m}$]')
axs1[0].set_ylabel(r'$p$ ['+conversion_pressure_unit+']')
axs1[1].set_xlabel(r'$x$ [$\mathrm{m}$]')
axs1[1].set_ylabel(r'$v$ [$\mathrm{m} / \mathrm{s}$]')
lo_00, = axs1[0].plot(pl_vec,pressure_conversion(p_old,initial_pressure_unit, conversion_pressure_unit),marker='.')
lo_01, = axs1[1].plot(pl_vec,v_old,marker='.')
axs1[0].autoscale()
axs1[1].autoscale()

fig1.tight_layout()
fig1.show()
plt.pause(1)


In [6]:
# loop through time steps of the pipeline
for it_pipe in range(1,pipe.nt+1):

    # if it_pipe == 250:
    #     V.set_influx(0.)

# for each pipeline timestep, execute nt_eRK4 timesteps of the reservoir code
    # set initial conditions for the reservoir time evolution calculted with e-RK4
    V.set_pressure(p_old[0])
    V.set_outflux(v_old[0]*area_outflux)
    # calculate the time evolution of the reservoir level within each pipeline timestep to avoid runaway numerical error
    for it_res in range(nt_eRK4):
        V.timestep_reservoir_evolution()                                                             
    level_vec[it_pipe] = V.get_current_level()                                                 

    # change the Leitapparatöffnung  based on the target value
    T1.update_LA(LA_soll_vec[it_pipe])
    T1.set_pressure(p_old[-1])

    LA_ist_vec[it_pipe] = T1.get_current_LA()

    # set boundary conditions for the next timestep of the characteristic method
    p_boundary_res[it_pipe] = V.get_current_pressure()
    v_boundary_tur[it_pipe] = 1/A_pipe*T1.get_current_Q()

    # the the boundary conditions in the pipe.object and thereby calculate boundary pressure at turbine
    pipe.set_boundary_conditions_next_timestep(p_boundary_res[it_pipe],v_boundary_tur[it_pipe])
    p_boundary_tur[it_pipe] = pipe.get_current_pressure_distribution()[-1]
    v_boundary_res[it_pipe] = pipe.get_current_velocity_distribution()[0]

    # perform the next timestep via the characteristic method
    pipe.timestep_characteristic_method()

    # prepare for next loop
    p_old = pipe.get_current_pressure_distribution()
    v_old = pipe.get_current_velocity_distribution()

    # plot some stuff
        # remove line-objects to autoscale axes (there is definetly a better way, but this works ¯\_(ツ)_/¯ )
    lo_00.remove()
    lo_01.remove()
    # lo_02.remove()
        # plot new pressure and velocity distribution in the pipeline
    lo_00, = axs1[0].plot(pl_vec,pressure_conversion(p_old,initial_pressure_unit, conversion_pressure_unit),marker='.',c='blue')
    lo_01, = axs1[1].plot(pl_vec,v_old,marker='.',c='blue')
    # lo_02, = axs1[2].plot(level_vec_2,c='blue')
    fig1.suptitle(str(round(t_vec[it_pipe],2))+ ' s / '+str(round(t_vec[-1],2)) + ' s' )
    fig1.canvas.draw()
    fig1.tight_layout()
    fig1.show()
    plt.pause(0.001)    

        
        

In [7]:
print(V.get_current_influx())

0.7727272727272726


In [8]:
# plot time evolution of boundary pressure and velocity as well as the reservoir level

fig2,axs2 = plt.subplots(3,2)
axs2[0,0].set_title('Pressure reservoir')
axs2[0,0].plot(t_vec,pressure_conversion(p_boundary_res,initial_pressure_unit, conversion_pressure_unit))
axs2[0,0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[0,0].set_ylabel(r'$p$ ['+conversion_pressure_unit+']')

axs2[0,1].set_title('Velocity reservoir')
axs2[0,1].plot(t_vec,v_boundary_res)
axs2[0,1].set_ylim(-2*Q_nenn,+2*Q_nenn)
axs2[0,1].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[0,1].set_ylabel(r'$v$ [$\mathrm{m}/\mathrm{s}$]')

axs2[1,0].set_title('Pressure turbine')
axs2[1,0].plot(t_vec,pressure_conversion(p_boundary_tur,initial_pressure_unit, conversion_pressure_unit))
axs2[1,0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[1,0].set_ylabel(r'$p$ ['+conversion_pressure_unit+']')

axs2[1,1].set_title('Velocity turbine')
axs2[1,1].plot(t_vec,v_boundary_tur)
axs2[1,1].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[1,1].set_ylabel(r'$v$ [$\mathrm{m}/\mathrm{s}$]')

axs2[2,0].set_title('Level reservoir')
axs2[2,0].plot(t_vec,level_vec)
axs2[2,0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[2,0].set_ylabel(r'$h$ [m]')

axs2[2,1].set_title('LA')
axs2[2,1].plot(t_vec,100*LA_soll_vec)
axs2[2,1].plot(t_vec,100*LA_ist_vec)
axs2[2,1].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[2,1].set_ylabel(r'$LA$ [%]')
fig2.tight_layout()
plt.show()

In [9]:
V.get_info(full=True)
V.set_outflux(-10.)
for i in range(10):
    V.level = V.update_level(10.)
    print(V.get_current_level())

The cuboid reservoir has the following attributes: 
----------------------------- 
Base area             =       74.0       m² 
Outflux area          =       0.636      m² 
Current level         =       8.020201323491174 m
Critical level low    =       0.0        m 
Critical level high   =       inf        m 
Volume in reservoir   =       --         m³ 
Current influx        =       0.7727272727272726 m³/s 
Current outflux       =       -3.529572535198196 m³/s 
Current outflux vel   =       -5.548     m/s 
Current pipe pressure =       0.479      bar 
Simulation timestep   =       0.0005065  s 
Density of liquid     =       1000       kg/m³ 
----------------------------- 

9.47597527926513
10.931749235039087
12.387523190813043
13.843297146587
15.299071102360957
16.75484505813491
18.210619013908868
19.666392969682825
21.12216692545678
22.577940881230738
